#### Marketing Mix Modeling: A/B Testing marketing campaigns for revenue

This model is part of a series of Marketing Mix Models.  The retail client for whom the testing was done ran multiple marketing campaigns, which in some cases resulted in noticeably different revenue. Campaigns were run for approximately the same time and were of the same duration.  The question we tried to answer was if it was truly the message (and offer) of campaigns that made some of them more compelling than others. That is, the purpose is to find out if the difference in revenue produced by two different marketing campaigns is statistically significant, i.e., not due just to random chance. 

For this project, I tested 26 segements of customer population. The segements, reflecting geographical, demographic and financial information, were randomly split into two groups: one group was receiving offer A and another group receiving offer B. Then, to compare the differences in resulting revenues, I ran a T-test for the difference of the means.

In [13]:
import pandas as pd
import numpy as np
from scipy import stats as st
from math import sqrt

##### 1. Read the data

In [69]:
df = pd.read_csv("T-Test Data.csv")

In [71]:
df.head()

,Rev_A,Rev_B
0,10304.369941,14333.879902
1,9759.209913,10107.659952
2,12420.849965,7452.569997
3,9672.369977,11695.009921
4,13675.789897,26398.599864


##### 2. Compute Mean and Variance

In [75]:
# Print out Means for FS70 and FS60 Revenues:
mean1,mean2 = np.mean(df, axis = 0)
print(f'Mean of Campaign_A Revenue: ${mean1:.2f}\nMean of Campaign_B Revenue: ${mean2:.2f}')

Mean of Campaign_A Revenue: $12790.68
Mean of Campaign_B Revenue: $19798.09


In [95]:
# Print out Variances for Rev_A and Rev_B; set delta degree of freedom (ddof) to 1 as we calculate the mean variance 
# of sample, not the whole population.
var1,var2 = np.var(df, axis = 0, ddof = 1)
print(f'Variance of Campaign_A Revenue: ${var1:.2f}\nVariance of Campaign_B Revenue: ${var2:.2f}')


Variance of Campaign_A Revenue: $7531209.65
Variance of Campaign_B Revenue: $79452509.61


Since variances for Campaign_A and Campaign_B are significantly unequal, to compare the means, I will use Welch's test (T-test for samples with unequal variance).


##### 3. Run Welch's Test

In [163]:
# Run T-test to compare Rev_A and Rev_B
# testing for two-sided alternative (mean of Rev_B either less or greater than that of Rev_A)
t_statistic,p_value = st.ttest_ind(df['Rev_A'], df['Rev_B'], equal_var = False, alternative = 'two-sided') 

print(f'Test Statistic: {t_statistic:.6f}')
print(f'P-value: {p_value:.6f}')

Test Statistic: -3.831120
P-value: 0.000614


Compute pooled degrees of freedom for variables with unequal variance, find the critical t-value and evaluate test statistic:

$$Pooled.Degrees.Of.Freedom =\frac{(var1/n + var2/n)^2}{\frac{(var1/n)^2}{n-1} + \frac{(var2/n)^2}{n-1}}$$, where $n$ is the sample size, and $var1$ and $var2$ are variances of the variables.


In [130]:
# compute degrees of freedom for Rev_A and Rev_B, where n is a sample size:
n = df.shape[0]    ## sample size
alpha = 0.05

d_of_f = (var1/n + var2/n)**2/((var1/n)**2/(n-1) +(var2/n)**2/(n-1))

print(f'Pooled degrees of freedom: {d_of_f:.2f}')

# find T-critical value for two-tailed test with significance level alpha = 0.5, degrees of freedom found above (d_of_f)
t_critical = st.t.ppf(q = 1- alpha/2, df = d_of_f)
print(f'Critical t-value: {t_critical:.6f}')
print(f'Alpha: {alpha:.2f}')

Pooled degrees of freedom: 29.70
Critical t-value: 2.043146
Alpha: 0.05


The absolute value of our test statistic is greater than the critical t-value.  The probability associated with it (p-value of the test) is very small, much smaller than $alpha$, indicating the difference is statistically significant.

##### 4. Compute Confidence Intervals

Use critical t-value to compute margin and find confidence intervals for the effect (difference in means):
$$st.error = \sqrt{\frac{var1}{n} + \frac{var2}{n}}$$
$$margin = st.error * t.critical$$
$$CFhigh = effect + margin$$
$$CFlow = effect - margin$$


In [179]:
# Compute difference in means:
effect = mean2 - mean1

# Compute pooled standard error:
st_error = sqrt(var1/n + var2/n)

# Compute margin:
margin = st_error * t_critical

#Compute Confidence Intervals
CI_low = effect - margin
CI_hi = effect + margin


In [181]:
# Intialize empty frame with column names:
col_names = ['T-Statistic','P-value','T-Critical','Alpha','Effect', 'Lower CI','Higher CI']
test_df = pd.DataFrame(columns = col_names)


In [183]:
# add records to the data frame using .loc method - adding to the next row, which is gotton by using len(df)
test_df.loc[len] = [round(t_statistic,4),round(p_value,4),round(t_critical,4), alpha,round(effect,2), round(CI_low,2), round(CI_hi,2)]
test_df

,T-Statistic,P-value,T-Critical,Alpha,Effect,Lower CI,Higher CI
0,-3.8311,0.0006,2.0431,0.05,7007.42,3270.34,10744.49


##### 5. Conclusion
The effect - the difference of mean revenue for the two marketing campaigns - was NOT due to chance, i.e. it was statistically signficant.  The account manager was advised that using Campaign 'B' was resulting in a better response from the customer base.